In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import os
os.getcwd()

'D:\\DTI TUBES\\notebook'

In [3]:
import os, sys
sys.path.append('../')
os.chdir('../')

import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
from nltk.tokenize import TweetTokenizer

from utils.forward_fn import forward_sequence_classification
from utils.metrics import document_sentiment_metrics_fn
from utils.data_utils import DocumentCategoryDataset, DocumentCategoryDataLoader

In [4]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [5]:
# Set random seed
set_seed(26092020)

In [6]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = DocumentCategoryDataset.NUM_LABELS

# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [8]:
train_dataset_path = './dataset/news_category/train.csv'
valid_dataset_path = './dataset/news_category/valid.csv'
test_dataset_path = './dataset/news_category/test.csv'

In [9]:
train_dataset = DocumentCategoryDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = DocumentCategoryDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = DocumentCategoryDataset(test_dataset_path, tokenizer, lowercase=True)

train_loader = DocumentCategoryDataLoader(dataset=train_dataset, max_seq_len=256, batch_size=6, shuffle=True)  
valid_loader = DocumentCategoryDataLoader(dataset=valid_dataset, max_seq_len=256, batch_size=6,shuffle=False)  
test_loader = DocumentCategoryDataLoader(dataset=test_dataset, max_seq_len=256, batch_size=6, shuffle=False)

In [10]:
w2i, i2w = DocumentCategoryDataset.LABEL2INDEX, DocumentCategoryDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'tekno': 0, 'health': 1, 'business': 2}
{0: 'tekno', 1: 'health', 2: 'business'}


In [11]:
text = 'kesehatan ini sedang rugi karena teknologi jadul'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: kesehatan ini sedang rugi karena teknologi jadul | Label : business (44.444%)


In [12]:
optimizer = optim.Adam(model.parameters(), lr=3e-6)
model = model.cuda()

In [13]:
# Train
n_epochs = 3
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)
    
    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]        
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        
        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
        
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

  0%|                                                                                           | 0/34 [00:00<?, ?it/s]

(Epoch 1) TRAIN LOSS:0.3757 ACC:0.89 F1:0.89 REC:0.89 PRE:0.89 LR:0.00000300


VALID LOSS:0.1099 ACC:0.98 F1:0.98 REC:0.98 PRE:0.97: 100%|████████████████████████████| 34/34 [00:07<00:00,  4.61it/s]
  0%|                                                                                          | 0/167 [00:00<?, ?it/s]

(Epoch 1) VALID LOSS:0.1099 ACC:0.98 F1:0.98 REC:0.98 PRE:0.97


  0%|                                                                                           | 0/34 [00:00<?, ?it/s]

...

(Epoch 2) VALID LOSS:0.0738 ACC:0.98 F1:0.98 REC:0.98 PRE:0.98


  0%|                                                                                           | 0/34 [00:00<?, ?it/s]

(Epoch 3) TRAIN LOSS:0.0310 ACC:0.99 F1:0.99 REC:0.99 PRE:0.99 LR:0.00000300


VALID LOSS:0.0630 ACC:0.98 F1:0.98 REC:0.98 PRE:0.98: 100%|████████████████████████████| 34/34 [00:11<00:00,  2.98it/s]

(Epoch 3) VALID LOSS:0.0630 ACC:0.98 F1:0.98 REC:0.98 PRE:0.98


In [14]:
# Evaluate on test
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    _, batch_hyp, _ = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
df.to_csv('pred.txt', index=False)

print(df)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.52it/s]

    index     label
0       0     tekno
1       1  business
2       2  business
3       3    health
4       4  business
..    ...       ...
95     95     tekno
96     96    health
97     97    health
98     98     tekno
99     99    health

[100 rows x 2 columns]


In [15]:
text = 'IHSG sedang naik di level hijau'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: IHSG sedang naik di level hijau | Label : business (78.725%)


In [16]:
torch.save(model.state_dict(), './model/model_category.bin')